In [54]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key=os.getenv("GROQ_API_KEY")

In [55]:
from langchain_groq import ChatGroq

llm=ChatGroq(
    temperature=0.2,
    groq_api_key=api_key,
    model="llama-3.3-70b-versatile"
)

response=llm.invoke("The first person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [56]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://careers.nike.com/nike-stores-franchise-partner-manager/job/R-64538")
page_data=loader.load().pop().page_content
print(page_data)






















Nike Stores Franchise Partner Manager










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previ

In [57]:
from langchain_core.prompts import PromptTemplate
prompt_extract=PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM WEBSITE:
    {page_data}
    #INSTRUCTION:
    The scraped text is from the career's page of a website,
    Your job is to extract the job postings and return them in json format containing the following keys:
    'roles','exerience','skills' and 'description'.
    Only return the valid json.
    #VALID JSON(NO PREAMBLE):
    """
)

In [58]:
chain_extract=prompt_extract | llm
response=chain_extract.invoke(input={'page_data':page_data})
print(response.content)

```json
{
  "roles": "Nike Stores Franchise Partner Manager",
  "experience": "Strong background and expertise in working in Partners facing roles, real estate acumen, extensive local marketplace know-how, experience in managing Partners, driving fleet development, understanding retail KPIs",
  "skills": "Leadership, x-functional team and stakeholders management, contract management, retail and operations acumen, strong communication and presentation skills, business savviness",
  "description": "The Nike Stores Franchise Partner Manager will be the Primary Lead of Partner Business in France, ensuring Strategic Alignment and Business Development Plans, and obsessing Fleet Expansion & Performance. The role requires strong leadership, x-functional team and stakeholders management skills, knowledge of the marketplace and understanding of real estate dynamics, savyness in business and retail KPIs."
}
```


In [59]:
type(response.content)

str

In [60]:
#Converting it from string type to json format

from langchain_core.output_parsers import JsonOutputParser
json_parser=JsonOutputParser()
json_response=json_parser.parse(response.content)
print(json_response)

{'roles': 'Nike Stores Franchise Partner Manager', 'experience': 'Strong background and expertise in working in Partners facing roles, real estate acumen, extensive local marketplace know-how, experience in managing Partners, driving fleet development, understanding retail KPIs', 'skills': 'Leadership, x-functional team and stakeholders management, contract management, retail and operations acumen, strong communication and presentation skills, business savviness', 'description': 'The Nike Stores Franchise Partner Manager will be the Primary Lead of Partner Business in France, ensuring Strategic Alignment and Business Development Plans, and obsessing Fleet Expansion & Performance. The role requires strong leadership, x-functional team and stakeholders management skills, knowledge of the marketplace and understanding of real estate dynamics, savyness in business and retail KPIs.'}


In [61]:
type(json_response)

dict

In [62]:
import uuid
import chromadb

client=chromadb.PersistentClient("vectorstore") 
#Initializes a persistent Chroma client and stores the data in the local directory vectorstore.
#'vectorstore' is the directory where this persistent data will live.
collection=client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())])
        
                                 

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [64]:
links=collection.query(query_texts=jobs["skills"],n_results=2).get("metadatas")
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}]]

In [51]:
jobs=json_response
jobs['skills']

['Strong leadership skills',
 'X-functional team and stakeholders management skills',
 'Knowledge of the marketplace and understanding of real estate dynamics',
 'Savyness in business and retail KPIs',
 'Contract Management: negotiations, renewals, adjustments',
 'Retail & Operations acumen',
 'Strong communication and presentation skills',
 'Business savviness: ability to understand retail KPIs and productivity/profitability results, performance management and ability to identify commercial opportunities to drive sales']

In [66]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Raj, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Raj, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(json_response), "link_list": links})
print(res.content)

Subject: Enhancing Nike Stores Franchise Partner Management with AtliQ's Expertise

Dear Hiring Manager,

I came across the job description for Nike Stores Franchise Partner Manager, and I was impressed by the role's requirements, which align with AtliQ's capabilities in facilitating seamless business process integration through automated tools. As a business development executive at AtliQ, I'd like to highlight how our expertise can support the strategic alignment, business development plans, and fleet expansion & performance that this position demands.

At AtliQ, we have empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team's proficiency in leadership, cross-functional team and stakeholders management, contract management, retail and operations acumen, and strong communication and presentation skills can be a valuable asset to your organization.

To demonstrate our capabilities,